In [ ]:
molecule h2o2 {
  H     0.0000000000   0.9803530335  -0.8498671785
  O     0.0000000000   0.6988545188   0.0536419016
  O     0.0000000000  -0.6988545188   0.0536419016
  H     0.0000000000  -0.9803530335  -0.8498671785
}

# g_convergence needs to be tight for IRCs:
set g_convergence gau_verytight

set {
  basis                      dzp
  opt_type                   irc
  geom_maxiter               60
  irc_direction              backward
  d_convergence              9
  irc_points                 12
}

energy = frequencies('scf', dertype=1)

# Lower point group from C2v to C2
h2o2.reset_point_group('c2')

energy = optimize('scf')

In [ ]:
         C            1.040389109196     0.254964976384    -0.647674881834    12.000000000000
         C            0.689591079928     0.000000000000     0.708001346062    12.000000000000
         C           -0.689591079928     0.000000000000     0.708001346062    12.000000000000
         C           -1.040389109196    -0.254964976384    -0.647674881834    12.000000000000
         H            1.923694934595    -0.185448301950    -1.124148093391     1.007825032230
         H            0.711123863746     1.180002035639    -1.108938410294     1.007825032230
         H            1.381683913454    -0.238567611354     1.514789629143     1.007825032230
         H           -1.381683913454     0.238567611354     1.514789629143     1.007825032230
         H           -1.923694934595     0.185448301950    -1.124148093391     1.007825032230
         H           -0.711123863746    -1.180002035639    -1.108938410294     1.007825032230



In [1]:
# use psi4conda environment
import psi4
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import helpers as hp # Many useful functions from https://lcbc-epfl.github.io/iesm-public/intro.html

psi4.core.clean_options()

# The Z-matrix as a text string ### From previous optimization
data = """
       0 1
  H    -0.5958806528   0.9889214459   0.0000000000
  C    -0.5958806528  -0.1660941336   0.0000000000
  N     0.5535292657   0.0711607905   0.0000000000
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

#hp.drawXYZ(mol)
hp.drawXYZ_labeled(mol)




3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
ts = mol.clone()

psi4.core.clean_options()

output_file = "24_ring_closing_TS1.log"
psi4.set_memory("2GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

psi4.set_options({
        "OPT_TYPE": "ts",              # default => "min":  MIN, TS, IRC
        "FULL_HESS_EVERY": 0,         # default => -1 -> Does not do Hessian; 0 -> hessian at start
        "NORMAL_MODES_WRITE": True,    # default => False
         "g_convergence": "GAU_VERYTIGHT", # default => QCHEM:  MOLPRO, GAU, GAU_LOOSE, GAU_TIGHT, INTERFRAG_TIGHT, GAU_VERYTIGHT, TURBOMOLE, CFOUR, NWCHEM_LOOSE
    })


#energy_ts, hist = psi4.optimize("hf/sto-3g", molecule = ts, return_history=True)          
energy_ts, hist = psi4.optimize("b3pw91/6-31+G(d)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("b3lyp/6-311++G(d,p)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("B3LYP-d3bj2b/6-311++G(d,p)", molecule = ts, return_history=True)   


#print out TS in case something goes wrong
with open('ts.xyz', 'w') as f:
    f.write(ts.save_string_xyz_file())


  Memory set to   1.863 GiB by Python driver.
Optimizer: Optimization complete!


In [3]:
!ls -alr *.molden_normal_modes

-rw-r--r--  1 blink  staff  16932 25 May 20:56 irc.C4H6.20360.molden_normal_modes
-rw-r--r--  1 blink  staff  16932 22 May 22:26 butane_TS2.C4H6.16013.molden_normal_modes
-rw-r--r--  1 blink  staff  34528 15 May 15:57 butane_TS.default.29456.molden_normal_modes
-rw-r--r--  1 blink  staff  34528 14 May 22:34 butane_TS.default.1122.molden_normal_modes
-rw-r--r--  1 blink  staff  18963 22 May 22:01 butane_TS.C4H6.16013.molden_normal_modes
-rw-r--r--  1 blink  staff   1613 30 May 22:03 24_ring_closing_TS1.CHN.57124.molden_normal_modes
-rw-r--r--  1 blink  staff  16932 30 May 21:17 23_ring_closing_TS1.C4H6.56957.molden_normal_modes
-rw-r--r--  1 blink  staff  16932 30 May 21:27 23_Butane_PES_1.C4H6.56957.molden_normal_modes
-rw-r--r--  1 blink  staff   2937 30 May 20:55 22x_ring_closing_TS1.H2O2.56818.molden_normal_modes
-rw-r--r--  1 blink  staff   2061 30 May 21:09 22_Butane_PES_1.H2O2.56818.molden_normal_modes
-rw-r--r--  1 blink  staff  18963 28 May 21:48 10_ring_closing_TS1.C4H6.49499.

In [4]:
hp.show_normal_modes(filename='24_ring_closing_TS1.CHN.57124.molden_normal_modes')

interactive(children=(Dropdown(description='Normal mode:', options=((-1151.2298413019, 0), (-311.8748045197, 1…

In [8]:
output_file = "24_Butane_PES_1.out"
psi4.set_memory("4GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

irc_direction = "forward"

mol3 = ts.clone()

psi4.set_options({
     "geom_maxiter":500,
     "full_hess_every":1,
     "opt_type":"irc",
     "irc_step_size":0.25,
     "ensure_bt_convergence":True,
     "irc_direction":irc_direction,
     "reference": "rhf",
     "g_convergence": "GAU_VERYTIGHT", # default => QCHEM:  MOLPRO, GAU, GAU_LOOSE, GAU_TIGHT, INTERFRAG_TIGHT, GAU_VERYTIGHT, TURBOMOLE, CFOUR, NWCHEM_LOOSE
     "IRC_POINTS": 10
     })

E, history = psi4.optimize('b3pw91/6-31+G(d)',molecule=mol3, return_history=True)

#energy_ts, hist = psi4.optimize("hf/sto-3g", molecule = ts, return_history=True)          
#energy_ts, hist = psi4.optimize("b3pw91/6-31+G(d)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("b3lyp/6-311++G(d,p)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("B3LYP-d3bj2b/6-311++G(d,p)", molecule = ts, return_history=True)   
print("Done")


Optimizer: Optimization complete!
[{'step_number': 0, 'q': [2.2471476414711833, 2.249191383276396, 1.2500050559994194], 'x': [[-1.0429905356355837, 1.8620731975886977, 0.0], [-1.166671895501183, -0.3816681965544189, 0.0], [1.0035786713381203, 0.20898767257622156, 0.0]], 'f_q': [-8.613477276263412e-08, 1.6611731299999145e-07, 1.0033648075551781e-07], 'f_x': [-4.932370867173857e-08, -8.354667279812077e-08, 0.0, -1.2267826850790174e-07, 8.296724819760444e-08, 0.0, 1.720019771796403e-07, 5.794246005163298e-10, 0.0], 'energy': -93.3077457459735, 'q_pivot': [2.217790745962235, 2.2456977576572172, 1.3091600296593395], 'x_pivot': [[-1.0882099614665857, 1.8681165274821276, 0.0], [-1.1514089615367222, -0.3487735643505421, 0.0], [1.0335351632046614, 0.17004971047891473, 0.0]], 'step_dist': 0.0, 'arc_dist': 0.0, 'line_dist': 0.0}, {'step_number': 1, 'q': [2.1950509611988913, 2.2421882450597095, 1.3705048990584232], 'x': [[-1.135012207294134, 1.8775048147196995, 0.0], [-1.1341277374385115, -0.31754

In [9]:

traj = hp.mol2traj(mol3, history["coordinates"])
hp.drawXYZGeomSlider(traj)

interactive(children=(IntSlider(value=0, description='conf', max=29), Output()), _dom_classes=('widget-interac…

In [ ]:
# use psi4conda environment
import psi4

psi4.core.clean_options()

data = """
       0 1
  H    -0.5958806528   0.9889214459   0.0000000000
  C    -0.5958806528  -0.1660941336   0.0000000000
  N     0.5535292657   0.0711607905   0.0000000000
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

ts = mol.clone()
output_file = "HCN_test.log"
psi4.set_memory("2GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

psi4.set_options({
        "OPT_TYPE": "ts",              # default => "min":  MIN, TS, IRC
        "FULL_HESS_EVERY": 0,         # default => -1 -> Does not do Hessian; 0 -> hessian at start
        "NORMAL_MODES_WRITE": True,    # default => False
         "g_convergence": "GAU_VERYTIGHT", # default => QCHEM:  MOLPRO, GAU, GAU_LOOSE, GAU_TIGHT, INTERFRAG_TIGHT, GAU_VERYTIGHT, TURBOMOLE, CFOUR, NWCHEM_LOOSE
    })

energy_ts, hist = psi4.optimize("b3pw91/6-31+G(d)", molecule = ts, return_history=True)   


In [10]:
# use psi4conda environment
import psi4

psi4.core.clean_options()

# The Z-matrix as a text string ### From previous optimization
data = """
       0 1
  H    -0.5958806528   0.9889214459   0.0000000000
  C    -0.5958806528  -0.1660941336   0.0000000000
  N     0.5535292657   0.0711607905   0.0000000000
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

output_file = "HCN_test.log"
psi4.set_memory("4GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

irc_direction = "forward"

psi4.set_options({
     "geom_maxiter":500,
     "full_hess_every":1,
     "opt_type":"irc",
     "irc_step_size":0.25,
     "ensure_bt_convergence":True,
     "irc_direction":irc_direction,
     "reference": "rhf",
     "g_convergence": "GAU_VERYTIGHT", # default => QCHEM:  MOLPRO, GAU, GAU_LOOSE, GAU_TIGHT, INTERFRAG_TIGHT, GAU_VERYTIGHT, TURBOMOLE, CFOUR, NWCHEM_LOOSE
     "IRC_POINTS": 10
     })

E, history = psi4.optimize('b3pw91/6-31+G(d)',molecule=mol, return_history=True)

print("Done")

Optimizer: Optimization complete!
[{'step_number': 0, 'q': [2.182663115891963, 2.2178599492131337, 1.3672406132314339], 'x': [[-1.1260512372161497, 1.8687906923097142, 0.0], [-1.1260512372161497, -0.31387242358224887, 0.0], [1.0460187146336535, 0.13447440488303522, 0.0]], 'f_q': [0.002105864990347198, 0.03171575627271743, 0.02664603735574652], 'f_x': [-0.012208039418331125, 0.002105864990347198, 0.0, -0.021281638343718413, 0.003248956356406341, 0.0, 0.03348967776204954, -0.005354821346753539, 0.0], 'energy': -93.30878966958137, 'q_pivot': [2.159113406270059, 2.2142955079277304, 1.4289239260627964], 'x_pivot': [[-1.1711774776891843, 1.8763898043394096, 0.0], [-1.1084682272194104, -0.28181274663689193, 0.0], [1.0735619451099487, 0.09481561590798278, 0.0]], 'step_dist': 0.0, 'arc_dist': 0.0, 'line_dist': 0.0}, {'step_number': 1, 'q': [2.150118705955073, 2.2227529736250724, 1.4885615097356442], 'x': [[-1.2183684152227796, 1.8908137820696185, 0.0], [-1.0943701771533412, -0.2557264310765488,

In [11]:
traj = hp.mol2traj(mol, history["coordinates"])
hp.drawXYZGeomSlider(traj)

interactive(children=(IntSlider(value=0, description='conf', max=31), Output()), _dom_classes=('widget-interac…